In [11]:
"""
Also known as Inception v1.
InceptionNet are family of Networks, that are based on Inception Module.
The developers of the InceptionNet were the first to focus on increasing the width and the
depth of the model simultaneously to attain better accuracy while keeping the computing
resources constant.

The InceptionNet architecture focusses on parallel processing and 'extraction of various feature maps
concurrently'.

Inception Module:
The ideology behind the Inception module was that neurons that extract features together should learn together.
  ~ Hebbian's Principle
The architecture does not follow Sequential model approach where every operation such
as pooling or convolution is performed one after the other.



"""

"\nAlso known as Inception v1.\nInceptionNet are family of Networks, that are based on Inception Module.\nThe developers of the InceptionNet were the first to focus on increasing the width and the\ndepth of the model simultaneously to attain better accuracy while keeping the computing\nresources constant.\n\nThe InceptionNet architecture focusses on parallel processing and 'extraction of various feature maps\nconcurrently'.\n\nInception Module:\nThe ideology behind the Inception module was that neurons that extract features together should learn together.\n  ~ Hebbian's Principle\nThe architecture does not follow Sequential model approach where every operation such\nas pooling or convolution is performed one after the other.\n\n\n\n"

In [12]:
import torch

In [13]:
# load dataset here

In [14]:
class ConvBlock(torch.nn.Module):
  def __init__(self, in_channels, out_channels, k, s, p):
    super(ConvBlock, self).__init__()
    self.conv = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=k, stride=s, padding=p),
        torch.nn.ReLU()
    )

  def forward(self, x):
    return self.conv(x)

In [15]:
class ReduceConvBlock(torch.nn.Module):
  def __init__(self, in_channels, out_channels1, out_channels2, k, p):
    super(ReduceConvBlock, self).__init__()
    self.redConv = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels1, kernel_size=1, stride = 1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(in_channels=out_channels1, out_channels=out_channels2, kernel_size=k, stride=1, padding=p),
        torch.nn.ReLU()
    )

  def forward(self, x):
    return self.redConv(x)

In [16]:
class InceptionModule(torch.nn.Module):
  def __init__(self, in_channels, out_1x1, out_interm_3x3, out_3x3, out_interm_5x5, out_5x5, out_pool_proj):
    super(InceptionModule, self).__init__()
    self.conv1 = ConvBlock(in_channels=in_channels, out_channels=out_1x1, k=1, s=1, p=0)
    self.conv2 = ReduceConvBlock(in_channels=in_channels, out_channels1=out_interm_3x3, out_channels2=out_3x3, k=3, p=1)
    self.conv3 = ReduceConvBlock(in_channels=in_channels, out_channels1=out_interm_5x5, out_channels2=out_5x5, k=5, p=2)

    self.pool_proj = torch.nn.Sequential(
        torch.nn.MaxPool2d(kernel_size=3, stride=1),
        torch.nn.Conv2d(in_channels=in_channels, out_channels=out_pool_proj, kernel_size=1, stride=1),
        torch.nn.ReLU()
    )

  def forward(self, x):
    out1 = self.conv1(x)
    out2 = self.conv2(x)
    out3 = self.conv3(x)
    out4 = self.pool_proj(x)

    return torch.cat([out1, out2, out3, out4], dim=1) # dim=0 : cat along rows, dim=1 : cat along cols

In [17]:
class AuxClassifier(torch.nn.Module):
  def __init__(self, in_channels, num_classes):
    super(AuxClassifier, self).__init__()
    self.avgpool = torch.nn.AvgPool2d(kernel_size=5, stride=3)
    self.conv = torch.nn.Conv2d(in_channels=in_channels, out_channels=128, kernel_size=1)
    self.relu = torch.nn.ReLU()
    self.fc = torch.nn.Linear(4*4*128, 1024)
    self.dropout = torch.nn.Dropout(p=0.7)
    self.classifier = nn.Linear(1024, num_classes)

  def forward(self, inp):
    N = inp.shape[0]
    x = self.avgpool(inp)
    x = self.conv(x)
    x = self.relu(x)
    x = x.reshape(N, -1)
    x = self.fc(x)
    x = self.dropout(x)
    x = self.classifier(x)
    return x


In [19]:
class GoogleNet(torch.nn.Module):
  def __init__(self, in_channels = 3, num_class=1000):
    super(GoogleNet, self).__init__()
    self.conv1 = ConvBlock(in_channels=in_channels, out_channels=64, k=7, s=2, p=1)
    self.maxpool1 = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)


    self.conv2 = torch.nn.Sequential(
        ConvBlock(in_channels = 64, out_channels = 64, k = 1, s=1, p=0),
        ConvBlock(in_channels=64, out_channels = 192, k=3, s=1, p=1)
    )


    self.inception_3a = InceptionModule(in_channels=192, out_1x1=64, out_interm_3x3=96, out_3x3=128, out_interm_5x5=16, out_5x5=32, out_pool_proj=32)
    self.inception_3b = InceptionModule(256, 128, 128, 192, 32, 96, 64)
    # a MaxPool2d here
    self.inception_4a = InceptionModule(480, 192, 96, 208, 16, 48, 64)
    self.inception_4b = InceptionModule(512, 160, 112, 224, 24, 64, 64)
    self.inception_4c = InceptionModule(512, 128, 128, 256, 24, 64, 64)
    self.inception_4d = InceptionModule(512, 112, 144, 288, 32, 64, 64)
    self.inception_4e = InceptionModule(528, 256, 160, 320, 32, 128, 128)
    # a MaxPool2d here
    self.inception_5a = InceptionModule(832, 256, 160, 320, 32, 128, 128)
    self.inception_5b = InceptionModule(832, 384, 192, 384, 48, 128, 128)
    # a AvgPool2d here

    self.aux_classifier1 = AuxClassifier(512, num_class)
    self.aux_classifier2 = AuxClassifier(528, num_class)
    self.avgpool = nn.AdaptiveAvgPool2d(output_size=(7, 7))
    self.classifier = nn.Sequential(
        nn.Dropout(p=0.4),
        nn.Linear(1024 * 7 * 7, num_class)
    )

  def forward(self, input_img):
    N = input_img.shape[0]
    x = self.conv1(input_img)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool1(x)
    x = self.inception_3a(x)
    x = self.inception_3b(x)
    x = self.maxpool1(x)
    x = self.inception_4a(x)
    out1 = self.aux_classifier1(x)
    x = self.inception_4b(x)
    x = self.inception_4c(x)
    x = self.inception_4d(x)
    out2 = self.aux_classifier2(x)
    x = self.inception_4e(x)
    x = self.maxpool1(x)
    x = self.inception_5a(x)
    x = self.inception_5b(x)
    x = self.avgpool(x)
    x = x.reshape(N, -1)
    x = self.classifier(x)
    if self.training == True:
        return [x, out1, out2]
    else:
        return x

In [18]:
from torch.utils.tensorboard import SummaryWriter

if __name__ == '__main__':
    # Temporary define data and target
    batch_size = 5
    x = torch.randn((batch_size, 3, 224, 224))
    y = torch.randint(0,1000, (batch_size,))
    num_classes = 1000

    # Add to graph in tensorboard
    writer = SummaryWriter(log_dir='logs/googlenet')
    m = MyGoogleNet()
    # print(m)
    # we have x,o1,o2 = m(x)
    # m(x)[0] means x; m(x)[1] means o1; m(x)[2] means o2
    # o1 and o2 are output from auxclassifier
    print(m(x)[0].shape)
    m.eval()
    print(m.training)
    writer.add_graph(m, x)
    writer.close()

    # Notice here! When you going to train your network
    # Put these loss value into train step of your model

    m.train()

    """
    m.train() acctivates layers like Dropout() and BatchNorm(). without calling .train() these layers won't
    fired! modules like nn.Dropout check the model's mode internally using self.training
    """


    loss = nn.CrossEntropyLoss()
    loss1 = nn.CrossEntropyLoss()
    loss2 = nn.CrossEntropyLoss()
    discount = 0.3

    o,o1,o2 = m(x)

    total_loss = loss(o,y) + discount*(loss1(o1,y) + loss2(o2,y))
    print(total_loss)

    # And while inferencing the model, set the model into
    # model.eval() mode
    m.eval()